In [24]:
from google.colab import files
import pandas as pd

# Upload the file from your local desktop
# uploaded = files.upload()
# filename = "Medicaldataset_Mod4.csv"

# # Read the uploaded CSV file into a pandas DataFrame
# if filename in uploaded:
#   df_mod = pd.read_csv(filename)

#   # Display the first few rows of the DataFrame
#   print(f"Successfully loaded '{filename}':")
#   display(df_mod.head())
# else:
#   print(f"File '{filename}' not found in the uploaded files. Please ensure you uploaded the correct file.")

df_mod = pd.read_csv("Medicaldataset_Mod4.csv")

In [43]:
display(df_mod.head())

,Age,Gender,Heart rate,Systolic blood pressure,Diastolic blood pressure,Blood sugar,CK-MB,Troponin,Result,CK-MB_log,Troponin_log
0,64,1,66,160,83,160.0,1.80,0.012,negative,1.029619,0.011929
1,21,1,94,98,46,296.0,6.75,1.060,positive,2.047693,0.722706
2,55,1,64,160,77,270.0,1.99,0.003,negative,1.095273,0.002996
3,64,1,70,120,55,270.0,13.87,0.122,positive,2.699346,0.115113
4,55,1,64,112,65,300.0,1.08,0.003,negative,0.732368,0.002996


In [25]:
print(df_mod.info)


<bound method DataFrame.info of       Age Gender  Heart rate  Systolic blood pressure  \
0      64   Male          66                      160   
1      21   Male          94                       98   
2      55   Male          64                      160   
3      64   Male          70                      120   
4      55   Male          64                      112   
...   ...    ...         ...                      ...   
1314   44   Male          94                      122   
1315   66   Male          84                      125   
1316   45   Male          85                      168   
1317   54   Male          58                      117   
1318   51   Male          94                      157   

      Diastolic blood pressure  Blood sugar  CK-MB  Troponin    Result  
0                           83        160.0   1.80     0.012  negative  
1                           46        296.0   6.75     1.060  positive  
2                           77        270.0   1.99     0.003  ne

In [26]:
# Convert 'Gender' column to numerical representation
df_mod['Gender'] = df_mod['Gender'].map({'Male': 1, 'Female': 0})

In [27]:
# Assign each column as a variable
for column in df_mod.columns:
  globals()[column.replace(" ", "_").replace("-", "_")] = df_mod[column]

In [28]:
import numpy as np

# Define the columns to transform
cols_to_transform = ['CK-MB', 'Troponin']

# Apply logarithmic transformation, adding a small constant to avoid log(0)
for col in cols_to_transform:
    # Add a small constant (e.g., 1e-9) before applying log
    df_mod[col + '_log'] = np.log(df_mod[col] + 1)

# Display the first few rows with the new log-transformed columns
print("DataFrame with log-transformed columns:")
display(df_mod.head())

DataFrame with log-transformed columns:


,Age,Gender,Heart rate,Systolic blood pressure,Diastolic blood pressure,Blood sugar,CK-MB,Troponin,Result,CK-MB_log,Troponin_log
0,64,1,66,160,83,160.0,1.80,0.012,negative,1.029619,0.011929
1,21,1,94,98,46,296.0,6.75,1.060,positive,2.047693,0.722706
2,55,1,64,160,77,270.0,1.99,0.003,negative,1.095273,0.002996
3,64,1,70,120,55,270.0,13.87,0.122,positive,2.699346,0.115113
4,55,1,64,112,65,300.0,1.08,0.003,negative,0.732368,0.002996


In [29]:
import matplotlib.pyplot as plt
import math

In [34]:
# Convert 'Result' column to numerical representation
heartattack = df_mod.copy()
heartattack['Result'] = df_mod['Result'].map({'positive': 1, 'negative': 0})

In [36]:
import statsmodels.api as sm

X = df_mod[['Age', 'Gender', 'Heart rate', 'Systolic blood pressure', 'Diastolic blood pressure', 'Blood sugar', 'CK-MB_log', 'Troponin_log']]
y = df_mod['Result'].map({'positive': 1, 'negative': 0})


# Add constant
X = sm.add_constant(X)

# Fit model
logit_model = sm.Logit(y, X).fit()
print(logit_model.summary())

Optimization terminated successfully.
         Current function value: 0.372263
         Iterations 11
                           Logit Regression Results                           
Dep. Variable:                 Result   No. Observations:                 1319
Model:                          Logit   Df Residuals:                     1310
Method:                           MLE   Df Model:                            8
Date:                Sun, 29 Jun 2025   Pseudo R-squ.:                  0.4418
Time:                        14:17:24   Log-Likelihood:                -491.01
converged:                       True   LL-Null:                       -879.61
Covariance Type:            nonrobust   LLR p-value:                1.691e-162
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                       -5.4376      0.771     -7.057      0.000      -6.94

In [47]:
# Create a DataFrame from the logistic regression results
summary_df = pd.DataFrame({
    'coef': logit_model.params,
    'std err': logit_model.bse,
    'z': logit_model.tvalues,
    'P>|z|': logit_model.pvalues,
    'Lower CI': logit_model.conf_int()[0],
    'Upper CI': logit_model.conf_int()[1]
})

# Calculate Odds Ratios (OR) and their confidence intervals
summary_df['Odds Ratio (OR)'] = np.exp(summary_df['coef'])
summary_df['OR Lower CI'] = np.exp(summary_df['Lower CI'])
summary_df['OR Upper CI'] = np.exp(summary_df['Upper CI'])

# Display the summary table sorted by Odds Ratio
print("Logistic Regression Results (Sorted by Odds Ratio):")
display(summary_df.sort_values(by='Odds Ratio (OR)', ascending=False))

Logistic Regression Results (Sorted by Odds Ratio):


,coef,std err,z,P>|z|,Lower CI,Upper CI,Odds Ratio (OR),OR Lower CI,OR Upper CI
Troponin_log,33.550663,4.110625,8.161937,3.296941e-16,25.493986,41.607340,3.722783e+14,1.180042e+11,1.174460e+18
CK-MB_log,1.844510,0.151129,12.204842,2.928778e-34,1.548302,2.140718,6.324997e+00,4.703475e+00,8.505539e+00
Gender,0.254753,0.165343,1.540752,1.233772e-01,-0.069314,0.578820,1.290143e+00,9.330337e-01,1.783931e+00
Age,0.039357,0.006217,6.330517,2.443406e-10,0.027172,0.051543,1.040142e+00,1.027545e+00,1.052894e+00
Diastolic blood pressure,0.003678,0.006973,0.527531,5.978251e-01,-0.009988,0.017345,1.003685e+00,9.900618e-01,1.017496e+00
Heart rate,-0.000262,0.005213,-0.050230,9.599394e-01,-0.010479,0.009955,9.997382e-01,9.895759e-01,1.010005e+00
Blood sugar,-0.000786,0.001028,-0.764523,4.445559e-01,-0.002802,0.001229,9.992141e-01,9.972021e-01,1.001230e+00
Systolic blood pressure,-0.003146,0.003809,-0.825991,4.088093e-01,-0.010611,0.004319,9.968591e-01,9.894455e-01,1.004328e+00
const,-5.437609,0.770508,-7.057171,1.699260e-12,-6.947778,-3.927441,4.349870e-03,9.607678e-04,1.969401e-02
